# Исследование надежности заемщиков.

Цель проекта - определить какие факторы влияют на погашение кредита в срок. 

Задачи проекта: 
1. Изучить данные банка  - статистику о платежеспособности клиентов.
2. Проработать данные.
3. Подтвердить или опровергнуть:

•   Есть ли зависимость между количеством детей и возвратом кредита в срок?

•	Есть ли зависимость между семейным положением и возвратом кредита в срок?

•	Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

•	Как разные цели кредита влияют на его возврат в срок?

### Шаг 1. Обзор данных

Открытие данных и прочтение информации из выгрузки. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Файл содежит 21525 записей, 12 колонок с разным типом данных. Заголовки написаны по правилам, изменение не требуется. 
Имеются пропуски данных в снескольких столбцах: days_employed (общий трудовой стаж в днях) - 2174, total_income(ежемесячный доход) - 2174.


Данные c пропущенными значениями в двух столбцах относятся к количественным. Работа с пропусками в подобных случаях отличается от работы с пропусками в категориальных переменных. Как правило пропуски количественных переменнх заполняют характерными значениями по выборке. 

### Шаг 2.1 Заполнение пропусков

Для определения метода замены пропусков применяем методы описательной статистики. Минимум, максимум, среднее и медиану. 
Помимо этого проверяем другие столбцы на аномалии.  

In [3]:
data['days_employed'].min() #минимальный стажи

-18388.949900568383

Подробнее смотрим возникшую аномалию

In [4]:
data.loc[data['days_employed'] == data['days_employed'].min()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью


In [5]:
data['days_employed'].max() #максимальный стаж

401755.40047533

Подробнее смотрим возникшую аномалию по всем параметрам таблицы

In [6]:
data.loc[data['days_employed'] == data['days_employed'].max()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью


In [7]:
data['days_employed'].mean() #среднее значение 

63046.49766147338

In [8]:
data['days_employed'].median() #медиана

-1203.369528770489

Медианное значение по столбцу days_employed - отрицательное, что не характерно для данного показателя. Возможно, отрицательные значения возникли из-за разницы в учетных системах. Предположим, что трудовой стаж мужчины:47 лет (с 18 по 65). В год - количество рабочих дней примерно  - 247 дней. При переводе в часы получается 92 879 часов (47 лет * 247 раб. дней * 8 часовой день ). У женщин: 37 лет, 9 139 рабочих дней и 73 112 раб. часа.  
Тогда часть данных отражена в часах, а часть данных в днях. Например максимальное значение соответсвует 401 755 / 11 609 = 34,58 года, трудовой стаж женщин - 37 лет (данные в меньшую сторону могут быть обусловлены местом работы, выход на пенсию ранее, ну или это данные до реформы в пенсинной системе).

Поскольку предстоит проработка трудового стажа, на мой вгляд, сначала необходимо взглянуть на столбец возраста клиента, выявить в нем аномалии. Т.к. трудовой стаж напрмую зависит от возраста клиента. 

In [9]:
for old in data['dob_years'].unique(): # цикла замены 0 значений по медиане возраста
    median = data.loc[data['dob_years'] == old, 'dob_years'].median() 
    data.loc[(data['days_employed'].isna()) & (data['days_employed'] == old), 'days_employed'] = median

In [10]:
data.groupby('income_type')['dob_years'].median()

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

После проработки столбца с возрастом клиентов, прорабатывем данные с пропусками по  трудовому стажу. Применяя функцию abs() - 
для избавления от отрицательных значений. Применяя расчеты по средним данным ((11609+9139)/2) трудовому стажу для мужчин и женщин в днях.

In [11]:
data['days_employed'] = data['days_employed'].apply(abs) # значения по модулю, избавление от отрицательных значений 
data.loc[data['days_employed'].isna(), 'days_employed'] = data['days_employed'] > 10374 / 24


In [12]:
data['days_employed'].isna().sum() #проверка пустых значение 

0

прорабтка данных по столбцу доходы клиентов, а именно работа с пропусками. Просмотр данных описательной статистики. 

In [13]:
data['total_income'].min() #минимальный доход 

20667.26379327158

In [14]:
data['total_income'].max() #максимальный доход

2265604.028722744

In [15]:
data['total_income'].mean() #средний доход

167422.30220817294

In [16]:
data['total_income'].median() #медиана по доходу 

145017.93753253992

In [17]:
data.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Простой заменой по среднему  - пропуски в столбце total_income нельзя, т.к. большой разрыв между минимальным и максимальным значением, а среднее значение не соответсвует медиане. Мединное значение по всему столбцу не целесобразно брать, так как медиана предпринимателя далека от медианы пенсионера. Поэтому замена пропусков в столбце total_income необходимо заменить медианой по типу занятости.     

In [18]:
for income in data['income_type'].unique():  #при помощи цикла замена значений 0 по медиане дохода столбца тип занятости 
    median = data.loc[data['income_type'] == income, 'total_income'].median() 
    data.loc[(data['total_income'].isna()) & (data['income_type'] == income), 'total_income'] = median 

Работа с аномалиями столбца количество детей. В данных столбца имеются логические ошибки, - 1 ребенка не может быть. Дети/ребенок есть или детей нет. Предположительно, -1 указано случайно или это тире. 

In [19]:
data['children'] = data['children'].replace(-1, 1) #замена количества -1 детей на 1

In [20]:
data['children'].max() #максимальное количество детей 

20

Предположительно это опечатка, расположение клавиш находится рядом, оформлена замена на 2 ребенка. 

In [21]:
data['children'] = data['children'].replace(20, 2) #замена количества 20 детей на 2 

### Шаг 2.2 Проверка данных на аномалии и исправления.

Проверка данных частично была оформлена при заполнении пропусков, а именно, были устранены аномалии возрасту с 0 значениями, стажу - с "-" значением, количеству детей - с значением - 1 и 20. 

Просмотр столбца - пол, выявлена одна аноммалия, которая при удалении не повлияет на результаты. 

In [22]:
data['gender'].unique() 

array(['F', 'M', 'XNA'], dtype=object)

In [23]:
data.groupby('gender').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,income_type,debt,total_income,purpose
gender,,,,,,,,,,,
F,14236,14236,14236,14236,14236,14236,14236,14236,14236,14236,14236
M,7288,7288,7288,7288,7288,7288,7288,7288,7288,7288,7288
XNA,1,1,1,1,1,1,1,1,1,1,1


In [24]:
data.loc[data['gender'] == 'XNA']['gender'] #находим строкув которой находится значение 

10701    XNA
Name: gender, dtype: object

In [25]:
data = data.drop(index=[10701])#удаляем строчку

In [26]:
data['gender'].unique() #проверка строчку

array(['F', 'M'], dtype=object)

In [27]:
data['family_status'].unique() #проверяем даннные по семейному статусу 

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [28]:
data.loc[data['family_status'] == 'Не женат / не замужем']['family_status_id'] #находим строкув которой находится значение 

24       4
29       4
36       4
38       4
40       4
        ..
21480    4
21488    4
21500    4
21512    4
21514    4
Name: family_status_id, Length: 2813, dtype: int64

In [29]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

### Шаг 2.3. Изменение типов данных.

После замены пустых значений столбцы days_employed приобрел тип данных - object. Что в дальнейшем может привести к неожиданным результатам или неправильному результату. Для дальнейшего анализа нужно преобазовать тип.

In [30]:
data.dtypes

children              int64
days_employed        object
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [31]:
data['days_employed'] = data['days_employed'].astype('int') #преобразование через astype 

In [32]:
data['total_income'] = data['total_income'].astype('int') #преобразование через astype 

In [33]:
data.dtypes #проверка результата 

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Шаг 2.4. Удаление дубликатов.

При проверке других столбцов были выявлены неявные дублика (регистр) 

In [34]:
data['education'].unique()#просмотр уникальных значений 

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [35]:
data['education'] = data['education'].str.lower()

In [36]:
data['education'].unique()#проверка 

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [37]:
data['purpose'].unique() #просмотр уникальных значений 

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В данных по столбцу purpose  много однокренных слов, слов и выражений описывающих одно и то же. Возможно, при заявке на кредит нет введенной формы и каждый человек заполняет по образцу. Для упрощения, неявные дубликаты можно обединить.  

In [38]:
def purpose_category (cell):
    if  'авто' in cell:
        return 'операции с автомобилем'
    elif 'недвиж' in cell or 'жиль' in cell:
        return 'операции с недвижимостью'
    elif 'свадьб' in cell:
        return 'проведение свадьбы'
    elif 'образо' in cell:
        return 'получение образования'

data['purpose_category'] = data['purpose'].apply(purpose_category)

In [39]:
data[['purpose', 'purpose_category']].head(10)

,purpose,purpose_category
0,покупка жилья,операции с недвижимостью
1,приобретение автомобиля,операции с автомобилем
2,покупка жилья,операции с недвижимостью
3,дополнительное образование,получение образования
4,сыграть свадьбу,проведение свадьбы
5,покупка жилья,операции с недвижимостью
6,операции с жильем,операции с недвижимостью
7,образование,получение образования
8,на проведение свадьбы,проведение свадьбы
9,покупка жилья для семьи,операции с недвижимостью


In [40]:
data.duplicated().sum() #нашла количество дубликатов

71

In [41]:
data[data.duplicated()] #нашла предпросмотр

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
2849,0,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи,операции с недвижимостью
3290,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,проведение свадьбы
4182,1,0,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба,проведение свадьбы
4851,0,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба,проведение свадьбы
5557,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,0,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование,получение образования
21032,0,0,60,среднее,1,женат / замужем,0,F,пенсионер,0,118514,заняться образованием,получение образования
21132,0,0,47,среднее,1,женат / замужем,0,F,сотрудник,0,142594,ремонт жилью,операции с недвижимостью
21281,1,0,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости,операции с недвижимостью


In [42]:
data = data.drop_duplicates()#удаление

In [43]:
data.duplicated().sum()#нпроверка

0

### Шаг 2.5. Категоризация данных.

Определение зависимости между количеством детей и возвратом кредита в срок.

In [44]:
children_report = data.pivot_table(index = ['children'], values = 'debt', aggfunc = ['sum', 'count', 'mean']) 
#создание сводной таблицы для просмотра даннх в сокращенном виде по заданному столбцу
children_report.columns = ['Задолженность', 'Всего', '%']#создание заголовков для таблицы 
children_report = children_report.sort_values(by = ['Всего'], ascending = False) #фильтация
display(children_report.style.format({'%':'{:.2%}'})) #редактирование формата

,Задолженность,Всего,%
children,,,
0,1063,14090,7.54%
1,445,4855,9.17%
2,202,2128,9.49%
3,27,330,8.18%
4,4,41,9.76%
5,0,9,0.00%


Определение зависимости между семейным положением и возвратом кредита в срок. При проверкеданных установлено, что статусы женат / замужем - 0, гражданский брак - 1, вдовец / вдова - 2, в разводе - 3, Не женат / не замужем - 4

In [45]:
family_status_id_report = data.pivot_table(index = ['family_status_id'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
family_status_id_report.columns = ['Задолженность', 'Всего', '%']
family_status_id_report = family_status_id_report.sort_values(by = ['family_status_id'], ascending = True)
display(family_status_id_report.style.format({'%':'{:.2%}'}))

,Задолженность,Всего,%
family_status_id,,,
0,931,12339,7.55%
1,388,4150,9.35%
2,63,959,6.57%
3,85,1195,7.11%
4,274,2810,9.75%


In [46]:
data.groupby('family_status').count()#просмотр 

,children,days_employed,dob_years,education,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
family_status,,,,,,,,,,,,
Не женат / не замужем,2810,2810,2810,2810,2810,2810,2810,2810,2810,2810,2810,2810
в разводе,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195
вдовец / вдова,959,959,959,959,959,959,959,959,959,959,959,959
гражданский брак,4150,4150,4150,4150,4150,4150,4150,4150,4150,4150,4150,4150
женат / замужем,12339,12339,12339,12339,12339,12339,12339,12339,12339,12339,12339,12339


Определение зависимости между целю кредита возврат в срок.

In [47]:
new_purpose_report = data.pivot_table(index = ['purpose_category'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
new_purpose_report.columns = ['Задолженность', 'Всего', '%']
new_purpose_report = new_purpose_report.sort_values(by = ['Задолженность'], ascending = False)
display(new_purpose_report.style.format({'%':'{:.2%}'}))

,Задолженность,Всего,%
purpose_category,,,
операции с недвижимостью,782,10810,7.23%
операции с автомобилем,403,4306,9.36%
получение образования,370,4013,9.22%
проведение свадьбы,186,2324,8.00%


Определение зависимости между уровнем дохода и возвратом кредита в срок.

In [48]:
def total_income_category(income):
    
#условие:
        #0–30000 — 'E';
        #30001–50000 — 'D';
        #50001–200000 — 'C';
        #200001–1000000 — 'B';
        #1000001 и выше — 'A'. 
      
    if income >= 0 and income <= 30000:
        return 'E'
    elif income >= 30001 and income <= 50000:
        return 'D'
    elif income >= 50001 and income <= 200000:
        return 'C'
    elif income >= 200001 and income <= 1000000:
        return 'B'
    else:
        return 'A'
    
data['total_income_category'] = data['total_income'].apply(total_income_category)

In [49]:
data.head(5)#просмотр

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,B
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,C
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,C
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования,B
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы,C


In [50]:
data.groupby('total_income_category')['total_income'].median() #просмотр медианого значения для наглядности и выводов

total_income_category
A    1223042.0
B     257692.0
C     130395.0
D      43857.5
E      26431.0
Name: total_income, dtype: float64

In [51]:
data.groupby('total_income_category')['total_income'].count()

total_income_category
A       25
B     5041
C    16015
D      350
E       22
Name: total_income, dtype: int64

In [52]:
debt_pivot = data.pivot_table(index = ['total_income_category'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
debt_pivot.columns = ['Задолженность', 'Всего', '%']
debt_pivot = debt_pivot.sort_values(by = ['Задолженность'], ascending = False)
display(debt_pivot.style.format({'%':'{:.2%}'}))

,Задолженность,Всего,%
total_income_category,,,
C,1360,16015,8.49%
B,356,5041,7.06%
D,21,350,6.00%
A,2,25,8.00%
E,2,22,9.09%


Для создания портрета надежного и ненадежного заемщика рассмтрим задолженность по гендерному признаку 

In [53]:
gender_pivot = data.pivot_table( index = 'gender', 
                              columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
gender_pivot.columns = ['Нет задолженности', 'Задолженность']
gender_pivot['%'] = gender_pivot['Задолженность'] / (gender_pivot['Задолженность'] + gender_pivot['Нет задолженности'])
gender_pivot = gender_pivot.sort_values(by = ['Задолженность'], ascending = False)
display(gender_pivot.style.format({'%':'{:.2%}'}))

,Нет задолженности,Задолженность,%
gender,,,
F,13180,994,7.01%
M,6532,747,10.26%


Для создания портрета надежного и ненадежного заемщика рассмтрим задолженность по уровню образования 

In [54]:
education_pivot = data.pivot_table( index = 'education', 
                              columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
education_pivot.columns = ['Нет задолженности', 'Задолженность']
education_pivot['%'] = education_pivot['Задолженность'] / (education_pivot['Задолженность'] + education_pivot['Нет задолженности'])
education_pivot = education_pivot.sort_values(by = ['Задолженность'], ascending = False)
display(education_pivot.style.format({'%':'{:.2%}'}))

,Нет задолженности,Задолженность,%
education,,,
среднее,13808,1364,8.99%
высшее,4972,278,5.30%
неоконченное высшее,675,68,9.15%
начальное,251,31,10.99%
ученая степень,6,0,0.00%


In [55]:
data.loc[data['total_income_category'] == 'B'].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,5041.000000,5041.000000,5041.000000,5041.000000,5041.000000,5041.000000,5041.000000
mean,0.502083,42261.837135,42.341401,0.691529,0.952390,0.070621,286578.919064
std,0.771032,113805.944708,11.514186,0.593170,1.425155,0.256216,94892.470242
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,200001.000000
25%,0.000000,950.000000,33.000000,0.000000,0.000000,0.000000,224448.000000
50%,0.000000,2081.000000,42.000000,1.000000,0.000000,0.000000,257692.000000
75%,1.000000,4209.000000,51.000000,1.000000,1.000000,0.000000,314984.000000
max,5.000000,401674.000000,72.000000,4.000000,4.000000,1.000000,997014.000000


In [56]:
data.loc[data['total_income_category'] == 'B']['income_type'].value_counts()

сотрудник          2377
компаньон          1740
пенсионер           552
госслужащий         369
предприниматель       2
безработный           1
Name: income_type, dtype: int64

In [57]:
data.loc[data['total_income_category'] == 'B']['education'].value_counts()

среднее                2924
высшее                 1858
неоконченное высшее     217
начальное                40
ученая степень            2
Name: education, dtype: int64

In [58]:
data.loc[data['total_income_category'] == 'B']['family_status_id'].value_counts()

0    2961
1     975
4     667
3     282
2     156
Name: family_status_id, dtype: int64

In [59]:
data.loc[data['total_income_category'] == 'B']['children'].value_counts()

0    3245
1    1175
2     522
3      88
4       7
5       4
Name: children, dtype: int64

In [60]:
data.loc[data['total_income_category'] == 'B']['gender'].value_counts()

F    2711
M    2330
Name: gender, dtype: int64

In [61]:
data.loc[data['total_income_category'] == 'B']['purpose_category'].value_counts()

операции с недвижимостью    2568
операции с автомобилем      1031
получение образования        897
проведение свадьбы           545
Name: purpose_category, dtype: int64

### Ответы на вопросы.

#### Вопросы :

•	Есть ли зависимость между количеством детей и возвратом кредита в срок?

•	Есть ли зависимость между семейным положением и возвратом кредита в срок?

•	Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

•	Как разные цели кредита влияют на его возврат в срок?

#### Выводы:

Рассмотрев по очередно предположительные факторы, влияющие на погашение кредита в срок, пришла к выводам:

1. самая большая группа по количеству кредитов - это группа без детей. Но если брать во внимание ее численность, а это 65% от всей выборки, то процент просроченной задолженности внутри - 7.51% - не велик. Т.е. больше всего берут кредиты - без детей. Группы с детьми малочисленней, они по количеству уменьшаются, а процент невозвратов увеличивается. В то время, как самая малочисленная, но с самым большим не возвратом - группа с 4 детьми. 

2. самая большая группа по количеству кредитов - это группа с статусом женат / замужем, процент не возвратов в сравнении с другими невелик. Группы меньше по численности, но с большим процентом - группы гражданский брак и Не женат / не замужем.

3. по уровню дохода определено, что самая большпая группа по количеству кредитов и по просроченной заболженности - группа С, с медианным доходом - 130 453.5 (спарвелдиво заментить, что это медиана доходов по категориям). Наблюдается, что самые малочисленные группы, но  с самым большим процентом невозвратов внитри своей группы - это А (мадианое значение - 1 223 042) и Е (медианое значение - 26 431.0 )

4. по целям кредита самое большое число, как по количеству взятых, так и по количесту невозвратов - все операции с недвижимостью. Но в процентом соотношении в подгруппах самый большой невозврат - операции с автомобилем и обучение. 
 


## Общий вывод:

Проработав и рассмотрев статистику о платежеспособности клиентов банка, пришла к выводам. 
   Во-первых, что недостаточно представлена выборка по типу занатости - в декрете, студент, безработный, предприниматель. Данные являются не показательными и недостаточными, а именно слишком малое наблюдение (1 и 2 представителя). На мой взгляд это достаточно рисковые заемщики, их следовало бы исключить из общей выборки и анализа, чтобы не было выбросов. 
   Во-вторых, разделив выборку на категории по доходу, было отмечено, что категрии с самым большим доходом и с самым маленьким доходом - не многочислены, но с высоким процентом невозврата внутри своей ктегории. Самая многочисленная по кредитам и по задолженности - группа c средним уровнем дохода. 
   В-третьих, самыми значимыми, на мой взгляд являются фкторы, влияющие на возврат кредита в срок -наличие детей и их количество, а также семейный статус. Риск не возврата увеличивается, как и количество детей. А одинокие или в гражданском браке, не самые многочисленные, но с высоким процентом по невозврату. 
   В-четвертых, цель кредита, на мой взгляд, не самый определяющий фактор, но влияющий на возврат, самы частые опирации  операции с недвижимостью(самые "популярные", дорогостоящие и с самым большим количеством невозвратов), опять же самые большие по проценту внутри группы - это операции с автомобилем и получение образования.
 
 
Если за основной критерий взять - уровень дохода, то портрет ненадежного заемщика выгялит: 

в категориях E (доход медиана -  26431.0) и D (доход медиана -  43857.5)
Пенсионер (женщина) замужем, образование среднее/начальное/неоконченное высшее, цель - операции с недвижимостью. 
 
в категории С (доход медиана - 130395.0) 
Женщина с должностью сотрудник, замужем, образование среднее/начальное/неоконченное высшее, 2 детей, цель - операции с недвижимостью. 

в категории B  (доход медиана -   257692)
Мужчина с должностью сотрудник, женат, образование среднее/начальное/неоконченное высшее, от 2 и выше детей., цель - операции с недвижимостью. 